# Reading and Writing Data

pandas features a number of functions for reading tabular data as a DataFrame object. The most common are `pandas.read_csv` and `read_excel`. Full list of build-in format you can find [here](https://pandas.pydata.org/docs/reference/io.html).

In [ ]:
import pandas as pd
import numpy as np

You already know how to read a csv file.
It is a simple text file where columns are separated by comma.

In [ ]:
pov = pd.read_csv("C:\\Users\\iwo.augustynski\\Downloads\\share-of-population-in-extreme-poverty.csv")

Writing to the same format is easy:

In [ ]:
pov.to_csv("poverty.csv")

It works the same for Excel files

Writing:

In [ ]:
pov.to_excel("poverty.xlsx")

If you wish to specify sheet name:

In [ ]:
pov.to_excel("poverty_sheet.xlsx", sheet_name='Sheet_name_1')

If you wish to write to more than one sheet in the workbook, it is necessary to specify an ExcelWriter object:

In [ ]:
pov_copy = pov.copy()

with pd.ExcelWriter('poverty_sheets.xlsx') as writer:
    pov.to_excel(writer, sheet_name='Sheet_name_1')
    pov_copy.to_excel(writer, sheet_name='Sheet_name_2')

And reading:

In [ ]:
pov_xls = pd.read_excel("poverty.xlsx")

pov_xls = pd.read_excel("poverty_sheets.xlsx", sheet_name='Sheet_name_1') # or
pov_xls = pd.read_excel("poverty_sheets.xlsx", sheet_name= 0 ) 

For more options check `pd.read_excel?`

## Data Transformation

### Removing duplicates

In [ ]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

The DataFrame method duplicated returns a Boolean Series indicating whether **each row** is a duplicate (its columns values are exactly equal to those in an earlier row) or not:

In [ ]:
data.duplicated()

As you can see last row is identical to the row before.

Relatedly, `drop_duplicates` returns a DataFrame with rows where the `duplicated` array is False filtered out:

In [ ]:
data.drop_duplicates()

Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates: `data.duplicated(["name_of_column"])`

**Assignment**: create new dataframe pov_last consisting last year available for all countries.
Tip: use option keep="last" in `drop_duplicates()`

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

pov_last

### Pivoting

As you already know data in tables could be presented in two forms: "long" where each observation has its own row or "wide" where observations are located in columns. For instance our poverty dataset has the "long" format. Each observation i.e. share of population below poverty line for observed year has its own row.
In R you can swap formats using `tidyr` `pivot_longer` or `pivot_wider` functions.
Pandas offers obviously similar functionalities:

 - **Pivoting “Long” to “Wide” Format** - `pandas.pivot`
 
 - **Pivoting “Wide” to “Long” Format** - `pandas.melt` 

**Pivoting “Wide” to “Long” Format**

In [ ]:
# Our "wide" dataframe
data = pd.DataFrame(np.arange(6).reshape((3, 2)),
                    columns=pd.Index(["Africa", "Asia"], name="continent"))

data["country"] = ["one", "two", "three"]
data

In [ ]:
data_long = pd.melt(data, id_vars = "country")
data_long

**Pivoting “Long” to “Wide” Format**

In [ ]:
data_wide = data_long.pivot(index = "country", columns = "continent", values = "value") 
data_wide

**Assignment**: Pivot our `pov` dataFrame. Which column you can use?

First prepare our dataset. Filter only years >2010 and choose any three countries (you can use `isin` to select multiple values). Preferably in one go.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

pov_sample

Now pivot our pivot_sample!

In [ ]:

# YOUR CODE HERE
raise NotImplementedError()
pov_wide

As you can see it was necessary to create `NaN` values.

Sometimes it is better not to have `NaN` or something similar in dataset. In such cases these values are replaced by special number, preferably unrealistic like negative or higher than 100 in our case.

For that purpose pandas offers `pandas.fillna()` method.

In [ ]:
pov_wide1 = pov_wide.fillna(999)

pov_wide1

As you can see very conveniently missing values in all colums are replaced :) 
Unfortunately that means if you need to replace values only in selected columns you have to do it column by column in following way:
`your_dataframe['name_of_column'] = your_dataframe['name_of_column'].fillna(some_value)`

### Hierarchical Indexing

*Hierarchical indexing* is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Another way of thinking about it is that it provides a way for you to work with higher dimensional data in a lower dimensional form. Let’s start with a simple example: create a Series with a list of lists (or arrays) as the index:

In [ ]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

What you’re seeing is a prettified view of a Series with a `MultiIndex` as its index. The “gaps” in the index display mean “use the label directly above”:

In [ ]:
data.index

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely **select subsets** of the data:

In [ ]:
print(data["b"])
print(data["b":"c"])
print(data.loc[["b", "d"]])

Selection is even possible from an “inner” level. Here I select all of the values having the value 2 from the second index level:

In [ ]:
data.loc[:, 2]

**Hierarchical indexing plays an important role in reshaping data and in group-based operations like forming a pivot table.** 

For example, you can change "long" format to "wide" of a DataFrame using its `unstack` method:

In [ ]:
data.unstack()

The inverse operation of `unstack` is `stack`:

In [ ]:
data.unstack().stack()

With a DataFrame, **either axis can have a hierarchical index**:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Europe", "Europe", "Asia"],
                              ["Green", "Red", "Green"]])

frame

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["continent", "color"]

frame

You can see how many levels an index has by accessing its `nlevels` attribute:

In [ ]:
frame.index.nlevels

With partial column indexing you can similarly select groups of columns:

In [ ]:
frame["Europe"]

A `MultiIndex` can be created by itself and then reused; the columns in the preceding DataFrame with level names could also be created like this:

In [ ]:
pd.MultiIndex.from_arrays([["Europe", "Europe", "Asia"],
                          ["Green", "Red", "Green"]],
                          names=["continent", "color"])

As you know, `R` has row names instead of indexes. Contrary to the names of columns row names are rather useless.

That means in order to change shape of a table (from "long" to "wide" and vice versa) we just use columns. In pandas you must first transform columns to (hierarchical) index.

In [ ]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

frame

DataFrame’s `set_index` function will create a new DataFrame using one or more of its columns as the index:

In [ ]:
frame2 = frame.set_index(["c", "d"]) # By default, the columns are removed from the DataFrame, though you can leave them in by passing drop=False to set_index

frame2

`reset_index`, on the other hand, does the opposite of `set_index`; the hierarchical index levels are moved into the columns:

In [ ]:
frame2.reset_index()